In [1]:
!pip install xpander-sdk openai python-dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from xpander_sdk import XpanderClient
from openai import OpenAI
## Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()
xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")
nvidiaAPIKey = os.environ.get("NVIDIA_API_KEY", "")

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = nvidiaAPIKey
)

## Connecting to the agent
xpander_client = XpanderClient(api_key=xpanderAPIKey)
agent1 = xpander_client.agents.get(agent_id=xpanderAgentID)

agent1.get_tools()

b'\x1b\x1b!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software has not been tested with node v23.3.0.                                                                !!\x1b\x1b\n'
b'\x1b\x1b!!  Should you encounter odd runtime issues, please try using one of the supported release before filing a bug report.  !!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software is currently running on node v23.3.0.                                                                 !!\x1b\x1b\n'
b'\x1b\x1b!!  As of the current release of this software, supported node releases are:                                            !!\x1b

[{'type': 'function',
  'function': {'name': 'PgTavilyFetchInsightsFromTavilyAI',
   'description': 'This operation fetches insights from Tavily AI based on user queries related to xpander.ai. Use this operation when you need to gather detailed information, analyze content, or research specific aspects of xpander.ai. It is particularly useful for applications that require data-driven insights, competitive analysis, or in-depth research on digital platforms. IMPORTANT! Ensure to use body_params, query_params, path_params. These are crucial for correct function calling!',
   'parameters': {'type': 'object',
    'properties': {'queryParams': {'type': 'object',
      'properties': {},
      'required': []},
     'pathParams': {'type': 'object', 'properties': {}, 'required': []},
     'bodyParams': {'type': 'object',
      'properties': {'inputTask': {'type': 'string',
        'description': 'input user sub task based on the input query'}},
      'required': []}},
    'required': ['query_pa

In [13]:
import json
from xpander_sdk import LLMProvider

memory = []
memory.append({"role": "system", "content": "You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## together with the final answer"})
print(memory[-1])
memory.append({"role": "user", "content": "Get news about qwen2.5-coder"})
print(memory[-1])
number_of_calls = 1
memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
print(memory[-1])
for i in range(3):
    llm_response = client.chat.completions.create(
        model="meta/llama-3.1-70b-instruct",
        messages=memory,
        tools=agent1.get_tools(),
        tool_choice="required"
    )
    memory.append(llm_response.choices[0].message)
    print(memory[-1])
    if(llm_response.choices[0].message.tool_calls):
        print(llm_response.choices[0].message.tool_calls)
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump(), llm_provider=LLMProvider.NVIDIA_NIM)
        tool_response = agent1.run_tools(tools_to_run)
        for tool_response in tool_response:
            tool_result = {
                "Selected_Function_name": tool_response.function_name,
                "Http_Error_Code": tool_response.status_code,
                "Response_from_Target_System": json.dumps(tool_response.result),
                "Generated_Payload_From_AI": tool_response.payload,
                "Original_Tool_Id": tool_response.tool_call_id
            }
            print(tool_result)
            memory.append({"role": "tool", "content": json.dumps(tool_response.result), "tool_call_id": tool_response.tool_call_id})
print(memory[-1])


{'role': 'system', 'content': 'You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## together with the final answer'}
{'role': 'user', 'content': 'Get news about qwen2.5-coder'}
{'role': 'assistant', 'content': 'Step number: 1'}


KeyboardInterrupt: 